## Data Cleaning Function

In [1]:
import pandas as pd
#%load_ext lab_black

In [2]:
# load mortality the data

death03 = pd.read_csv(
    "../00_Source/US_VitalStatistics/Underlying Cause of Death, 2003.txt",
    error_bad_lines=False,
    sep="\t",
)
death04 = pd.read_csv(
    "../00_Source/US_VitalStatistics/Underlying Cause of Death, 2004.txt",
    error_bad_lines=False,
    sep="\t",
)
death05 = pd.read_csv(
    "../00_Source/US_VitalStatistics/Underlying Cause of Death, 2005.txt",
    error_bad_lines=False,
    sep="\t",
)
death06 = pd.read_csv(
    "../00_Source/US_VitalStatistics/Underlying Cause of Death, 2006.txt",
    error_bad_lines=False,
    sep="\t",
)
death07 = pd.read_csv(
    "../00_Source/US_VitalStatistics/Underlying Cause of Death, 2007.txt",
    error_bad_lines=False,
    sep="\t",
)
death08 = pd.read_csv(
    "../00_Source/US_VitalStatistics/Underlying Cause of Death, 2008.txt",
    error_bad_lines=False,
    sep="\t",
)
death09 = pd.read_csv(
    "../00_Source/US_VitalStatistics/Underlying Cause of Death, 2009.txt",
    error_bad_lines=False,
    sep="\t",
)
death10 = pd.read_csv(
    "../00_Source/US_VitalStatistics/Underlying Cause of Death, 2010.txt",
    error_bad_lines=False,
    sep="\t",
)
death11 = pd.read_csv(
    "../00_Source/US_VitalStatistics/Underlying Cause of Death, 2011.txt",
    error_bad_lines=False,
    sep="\t",
)
death12 = pd.read_csv(
    "../00_Source/US_VitalStatistics/Underlying Cause of Death, 2012.txt",
    error_bad_lines=False,
    sep="\t",
)
death13 = pd.read_csv(
    "../00_Source/US_VitalStatistics/Underlying Cause of Death, 2013.txt",
    error_bad_lines=False,
    sep="\t",
)
death14 = pd.read_csv(
    "../00_Source/US_VitalStatistics/Underlying Cause of Death, 2014.txt",
    error_bad_lines=False,
    sep="\t",
)
death15 = pd.read_csv(
    "../00_Source/US_VitalStatistics/Underlying Cause of Death, 2015.txt",
    error_bad_lines=False,
    sep="\t",
)

In [3]:
df = pd.concat(
    [death03, death04, death05, death06, death07, death08, death09, death10, death11, death12, death13, death14, death15],
    ignore_index=True,
)

In [4]:
def clean_mortality(df, State_Abbr):
    df[["County1", "State"]] = df.County.str.split(", ", expand=True)
    df = df[df["State"] == State_Abbr]
    df = df.drop(["Notes", "County", "County Code", "Year Code"], axis=1)
    df = df.rename(columns={"County1": "County"})
    df = df[(df["Drug/Alcohol Induced Cause Code"] == "D1") | (df["Drug/Alcohol Induced Cause Code"] == "D2") | (df["Drug/Alcohol Induced Cause Code"] == "D4") | (df["Drug/Alcohol Induced Cause Code"] == "D9")]
    df["Deaths"] = df["Deaths"].astype(float)
    df = df.groupby(["Year", "State", "County"], as_index=False).sum()
    df["Year"] = df["Year"].astype(int)
    return df

In [5]:
mortality = clean_mortality(df, "FL")
mortality

,Year,State,County,Deaths
0,2003,FL,Alachua County,11.0
1,2003,FL,Bay County,21.0
2,2003,FL,Brevard County,97.0
3,2003,FL,Broward County,230.0
4,2003,FL,Citrus County,16.0
...,...,...,...,...
453,2015,FL,Seminole County,63.0
454,2015,FL,St. Johns County,17.0
455,2015,FL,St. Lucie County,51.0
456,2015,FL,Sumter County,12.0


In [6]:
def clean_pop(df1,df2):
    df1 = df1.drop(["Unnamed: 1", 2000, 2001, 2002, "Unnamed: 12", "Unnamed: 13"], axis=1)
    df1 = df1.rename(columns={"Unnamed: 0": "County"})
    df2 = df2.drop(["Census", "Estimates Base", 2016, 2017, 2018, 2019], axis=1)
    df2 = df2.rename(columns={"Unnamed: 0": "County"})
    df1.drop(df1.head(1).index, inplace=True)
    df1.drop(df1.tail(8).index, inplace=True)
    df2.drop(df2.head(1).index, inplace=True)
    df2.drop(df2.tail(5).index, inplace=True)
    df1["County"] = df1["County"].str[1:]
    df2["County"] = df2["County"].str[1:]
    df1 = df1.melt(id_vars=["County"])
    df1 = df1.rename(columns={"variable": "Year", "value": "Population"})
    df1 = df1.groupby(["Year", "County"], as_index=False).sum()
    df2 = df2.melt(id_vars=["County"])
    df2 = df2.rename(columns={"variable": "Year", "value": "Population"})
    df2 = df2.groupby(["Year", "County"], as_index=False).sum()
    df2[["County1", "State"]] = df2.County.str.split(", ", expand=True)
    df2 = df2.drop(["County", "State"], axis=1)
    df2 = df2.rename(columns={"County1": "County"})
    df_concat = pd.concat([df1,df2],ignore_index=True)
    return df_concat

In [ ]:
fl_pop_pre_policy = pd.read_excel("../00_Source/Population/fl_pop_2000-2010.xls", header=[3])
fl_pop_post_policy = pd.read_excel("../00_Source/Population/fl_pop_2010-2019.xlsx", header=[3])

In [ ]:
pop = clean_pop(fl_pop_pre_policy,fl_pop_post_policy)
pop

## merge population data with mortality data

In [ ]:
def merge_mortalitypop(df_mortality, df_pop):
    merged = df_mortality.merge(df_pop, how="left")
    merged["Mortality Rate"] = merged["Deaths"] / merged["Population"]
    return merged

In [ ]:
merged = merge_mortalitypop(mortality, pop)

In [ ]:
merged

## Group by function (Calculate the total death and total population)

In [ ]:
def calc_avg_mortality(df1, df2, df3):
    df1 = df1.groupby('Year', as_index=False).sum()[['Year', 'Deaths', 'Population']]
    df2 = df2.groupby('Year', as_index=False).sum()[['Year', 'Deaths', 'Population']]
    df3 = df3.groupby('Year', as_index=False).sum()[['Year', 'Deaths', 'Population']]
    
    df = pd.concat([df1, df2, df3],ignore_index=True)
    df = df.groupby('Year', as_index=False).sum()
    df['Avg Mortality Rate'] = df['Deaths'] / df['Population']
    return df